In [1]:
begin
   using ModelingToolkit
    using AlgebraicPetri
    using AlgebraicPetri.Epidemiology
    using AlgebraicPetri.BilayerNetworks

    using Catlab, Catlab.Theories
    using Catlab.CategoricalAlgebra
    using Catlab.Graphics
    using Catlab.Graphics: Graphviz
    import Catlab.CategoricalAlgebra: migrate!
    using Catlab.WiringDiagrams
    using Catlab.Programs
    using Catlab.Programs.RelationalPrograms

    using Random
    using DifferentialEquations 
end

In [2]:
draw(d::WiringDiagram) = to_graphviz(d,
    orientation=LeftToRight,
    labels=true, label_attr=:xlabel,
    node_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    ),
    edge_attrs=Graphviz.Attributes(
      :fontname => "Courier",
    )
)

draw (generic function with 1 method)

In [3]:
function wf_load(f)
    return read_json_acset(LabelledReactionNet{Float64,Float64},f)
end

wf_load (generic function with 1 method)

In [4]:
function wf_get_dim(m)
    return nt(m)
end

wf_get_dim (generic function with 1 method)

In [5]:
function wf_mcopy(m)
    return pair(m,m)
end

wf_mcopy (generic function with 1 method)

In [6]:
function wf_get_rates(m)
    return m[:rate]
end

wf_get_rates (generic function with 1 method)

In [7]:
function wf_rand(s,num_s)
    rng = Random.seed!(s)
    v = rand(rng,num_s)
    return v
end

wf_rand (generic function with 1 method)

In [8]:
function wf_rate_add(m,v)
    m[:rate] = m[:rate] + v 
    return m
end

wf_rate_add (generic function with 1 method)

In [9]:
# @present Workflow(FreeSymmetricMonoidalCategory) begin
@present Workflow(FreeBiproductCategory) begin
    (File,LRN,Dim,Seed,RVect,TSpan,CExpr)::Ob 
    wf_load::Hom(File,LRN)
    # wf_mcopy::Hom(LRN,LRN⊗LRN)
    wf_get_dim::Hom(LRN,Dim)
    wf_get_rates::Hom(LRN,RVect)
    wf_rand::Hom(Seed⊗Dim,RVect)
    wf_rate_add::Hom(LRN⊗RVect,LRN)
    MTKCompile::Hom(LRN⊗TSpan,CExpr)
end

Presentation{ThBiproductCategory, Symbol}(Catlab.Theories.FreeBiproductCategory, (Ob = Catlab.Theories.FreeBiproductCategory.Ob{:generator}[File, LRN, Dim, Seed, RVect, TSpan, CExpr], Hom = Catlab.Theories.FreeBiproductCategory.Hom{:generator}[wf_load, wf_get_dim, wf_get_rates, wf_rand, wf_rate_add, MTKCompile]), Dict(:RVect => (:Ob => 5), :LRN => (:Ob => 2), :wf_rand => (:Hom => 4), :MTKCompile => (:Hom => 6), :wf_rate_add => (:Hom => 5), :wf_get_rates => (:Hom => 3), :TSpan => (:Ob => 6), :Dim => (:Ob => 3), :CExpr => (:Ob => 7), :wf_load => (:Hom => 1)…), Pair[])

In [10]:
load_perturb_sim = @program Workflow (f::File,s::Seed,ts::TSpan) begin # 
    m = wf_load(f)
    # m1, m2 = wf_mcopy(m)
    n_param = wf_get_dim(m)
    v = wf_rand(s,n_param)
    # v = wf_get_rates(m)
    m_perturb = wf_rate_add(m,v)
    sim_expr = MTKCompile(m_perturb,ts)
    return sim_expr # m_perturb # 
end

WiringDiagram{ThBiproductCategory}([:File,:Seed,:TSpan], [:CExpr], 
[ -2 => {inputs},
  -1 => {outputs},
  1 => Box(:wf_load, [:File], [:LRN]),
  2 => Box(:wf_get_dim, [:LRN], [:Dim]),
  3 => Box(:wf_rand, [:Seed,:Dim], [:RVect]),
  4 => Box(:wf_rate_add, [:LRN,:RVect], [:LRN]),
  5 => Box(:MTKCompile, [:LRN,:TSpan], [:CExpr]) ],
[ Wire((-2,1) => (1,1)),
  Wire((-2,2) => (3,1)),
  Wire((-2,3) => (5,2)),
  Wire((1,1) => (2,1)),
  Wire((2,1) => (3,2)),
  Wire((3,1) => (4,2)),
  Wire((4,1) => (5,1)),
  Wire((1,1) => (4,1)),
  Wire((5,1) => (-1,1)) ])

In [11]:
make_depvar(p,t) = :($p($t))
# function MTKCompile(bn::Union{AbstractLabelledBilayerNetwork, AbstractBilayerNetwork})
function MTKCompile(lrxn::AbstractLabelledReactionNet,tspan)    
    r = lrxn[:rate]
    c = lrxn[:concentration]

    lpn = LabelledPetriNet(lrxn);
    bn = LabelledBilayerNetwork();
    migrate!(bn,lpn);
    
    varstmt = :(@variables t)
    @show varnames = bn[:variable]
    append!(varstmt.args, make_depvar.(bn[:variable], :t))
    
    paramstmt = :(@parameters)
    params = bn[:parameter]
    append!(paramstmt.args, bn[:parameter])
    
    diffstmt = :(D = Differential(t))
  
    ϕs = map(parts(bn, :Box)) do b
      vars = map(incident(bn, b,:call)) do i
        j = bn[i, :arg]
        return bn[j, :variable]
      end
      p = :(*($(bn[b, :parameter])))
      append!(p.args, vars)
      return :($(Symbol("ϕ$b")) = $p)
    end
  
    
    infs = map(parts(bn, :Qout)) do tv
      vars = map(incident(bn, tv, :infusion)) do wa
        j = bn[wa, :influx]
        return Symbol("ϕ$j")
      end
      p = :(+())
      append!(p.args, vars)
  
      # same for the outfluxes
      vars = map(incident(bn, tv, :effusion)) do wn
        j = bn[wn, :efflux]
        return :(- $(Symbol("ϕ$j")))
      end
      append!(p.args, vars)
      return p
    end
  
    zparts = zip(bn[:tanvar], infs)
  
    eqns = [:(D($tanvar) ~ $rhs) for (tanvar, rhs) in zparts]
    eq = :([])
    append!(eq.args, eqns)
    eqnstmt = :(eqs = $eq)
  
    varnameexpr = Expr(:tuple, varnames...)
    parnameexpr = Expr(:tuple, params...)

    # sys = ODESystem(eq, t, name=:PetriNet)
    # prob = ODEProblem(sys, c, tspan, r)
    # sol = solve(prob,Tsit5())
  
    # prob = ODEProblem(MakeReactionSystem(model), u0, tspan, p)
    # sol = solve(prob, Tsit5(), tstops=sample_times)

    return quote
      $varstmt
      $paramstmt
      $diffstmt
      $(ϕs...)
      $eqnstmt
      return $varnameexpr, $parnameexpr, solve(ODEProblem(ODESystem(eqs, t, name=:PetriNet), $c, $tspan, $r))
    end
  
end

MTKCompile (generic function with 1 method)

In [15]:
# serialize program wiring diagram
write_json_acset(load_perturb_sim.diagram, "sim-with-perturb_dwd.json")

1359

In [16]:
# visualize simulation plan
draw(load_perturb_sim)

Graph("G", true, "dot", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Subgraph("", Catlab.Graphics.Graphviz.Statement[Catlab.Graphics.Graphviz.Node("n0in1", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in1")), Catlab.Graphics.Graphviz.Node("n0in2", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in2")), Catlab.Graphics.Graphviz.Node("n0in3", OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:id => "in3")), Catlab.Graphics.Graphviz.Edge(Catlab.Graphics.Graphviz.NodeID[Catlab.Graphics.Graphviz.NodeID("n0in1", "", ""), Catlab.Graphics.Graphviz.NodeID("n0in2", "", ""), Catlab.Graphics.Graphviz.NodeID("n0in3", "", "")], OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}())], OrderedCollections.OrderedDict{Symbol, Union{String, Catlab.Graphics.Graphviz.Html}}(:rank => "source", :rankdir => "TB"), OrderedCollections.Ord

In [17]:
# generate Julia program that executes simulation plan
wf_hom_expr = to_hom_expr(FreeBiproductCategory,load_perturb_sim)
wf_jfunc = Catlab.Programs.GenerateJuliaPrograms.compile(wf_hom_expr)

function = (x1, x2, x3;) -> begin
    begin
        v1 = (Main).wf_load(x1)
        v2 = (Main).wf_get_dim(v1)
        v3 = (Main).wf_rand(x2, v2)
        v4 = (Main).wf_rate_add(v1, v3)
        v5 = (Main).MTKCompile(v4, x3)
        return v5
    end
end

In [18]:
# expected output
#=  === wf_jfunc == 
function = (x1, x2, x3;) -> begin
    begin
        v1 = (Main).wf_load(x1)
        v2 = (Main).wf_get_dim(v1)
        v3 = (Main).wf_rand(x2, v2)
        v4 = (Main).wf_rate_add(v1, v3)
        v5 = (Main).MTKCompile(v4, x3)
        return v5
    end
end
=#

In [19]:
# apply plan to lrxnet from MIRA integration demo
wf_script = wf_jfunc(joinpath(@__DIR__, ".", "lrxnet_Mira_TC_est.json"),1234,(0,50))

varnames = bn[:variable] = [:S_City1, :E_City1, :I_City1, :A_City1, :SQ_City1, :H_City1, :R_City1, :EQ_City1, :D_City1, :S_City2, :E_City2, :I_City2, :A_City2, :SQ_City2, :H_City2, :R_City2, :EQ_City2, :D_City2]


quote
    #= In[11]:67 =#
    #= In[11]:11 =# @variables t S_City1(t) E_City1(t) I_City1(t) A_City1(t) SQ_City1(t) H_City1(t) R_City1(t) EQ_City1(t) D_City1(t) S_City2(t) E_City2(t) I_City2(t) A_City2(t) SQ_City2(t) H_City2(t) R_City2(t) EQ_City2(t) D_City2(t)
    #= In[11]:68 =#
    #= In[11]:15 =# @parameters strata_travel12 strata_travel12 strata_travel12 strata_travel12 strata_travel12 strata_travel21 strata_travel21 strata_travel21 strata_travel21 strata_travel21 unspook_s_disease prog_ei_disease prog_ea_disease hosp_i_disease recov_i_disease recov_a_disease recov_h_disease death_i_disease death_h_disease hosp_eq_disease expos_a_infect spook_a_infect espook_a_infect expos_i_infect spook_i_infect espook_i_infect unspook_s_disease prog_ei_disease prog_ea_disease hosp_i_disease recov_i_disease recov_a_disease recov_h_disease death_i_disease death_h_disease hosp_eq_disease expos_a_infect spook_a_infect espook_a_infect expos_i_infect spook_i_infect espook_i_infect
    #= In[11]:69 =#
 

In [ ]:
# Run that simulation!
# wf_vars, wf_params, wf_ode_sol = eval(wf_script);